In [66]:
import pandas as pd
import os
import base64
import json
from requests import post, get

#for privacy matters, i won't share here my client id and client secret.
# client_id = "????""
#client_secret = "???????"

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def get_artist_genres(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    
    result = get(query_url, headers=headers)
    
    if result.status_code == 200:
        json_result = json.loads(result.content)
        
        if 'artists' in json_result and 'items' in json_result['artists']:
            artist_data = json_result['artists']['items']
            if artist_data:
                artist = artist_data[0]
                genres = artist.get('genres', [])
                if genres:
                    return ", ".join(genres)
                else:
                    return "Unknown Genre"
            else:
                return "Unknown Genre"
        else:
            return "Unknown Genre"
    else:
        return "API Error"

# Load the Excel sheet containing the artist names
excel_file_path = '/Users/gabin/Documents/MUSIC DASHBOARD/deezer_data.xlsx'
df = pd.read_excel(excel_file_path, sheet_name="10_listeningHistory")

# Strip any extra spaces from column names
df.columns = df.columns.str.strip()

# Extract distinct artist names
distinct_artists = df['Artist_name'].unique()

# Proceed with genre retrieval for distinct artists
token = get_token()
artist_genres = {}

# Fetch genres for each distinct artist
for artist_name in distinct_artists:
    genres = get_artist_genres(token, artist_name)
    artist_genres[artist_name] = genres
    print(f"Artist: {artist_name}, Genres: {genres}")

# map the artist name to the genres
df['Genres'] = df['Artist_name'].map(artist_genres)

# Save the updated dataframe back to the Excel file
output_file_path = 'updated_artist_genres.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Updated Excel sheet saved as: {output_file_path}")

Artist: Supertramp, Genres: album rock, art rock, classic rock, glam rock, mellow gold, piano rock, progressive rock, rock, soft rock, symphonic rock
Artist: The Police, Genres: album rock, classic rock, permanent wave, rock
Artist: René Aubry, Genres: compositional ambient
Artist: Melanie De Biasio, Teun Verbruggen, Axel Gilain, Pascal Paulus, Pascal Mohy, Genres: belgian jazz, belgian pop
Artist: Drake, Genres: canadian hip hop, canadian pop, hip hop, pop rap, rap
Artist: Sexion d'Assaut, Genres: french hip hop, french pop, pop urbaine, rap conscient
Artist: Fonky Family, Genres: french hip hop, old school rap francais, rap conscient, rap marseille
Artist: L'entourage, Genres: french hip hop, rap inde
Artist: Creed, Genres: alternative metal, nu metal, post-grunge, rock
Artist: Jem, Genres: deep acoustic pop, lilith
Artist: Serge Gainsbourg, Genres: chanson, french rock
Artist: Caballero, Jeanjass, Le Seize, Genres: belgian hip hop, french hip hop
Artist: Booba, Genres: french hip ho